In [ ]:
import pylab as plt
import matplotlib as mpl
import numpy as np 
import tables
import tifffile
import glob
import functools

%matplotlib qt
%load_ext autoreload
%autoreload 2

mpl.rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
mpl.rcParams['xtick.major.pad']=8
mpl.rcParams['xtick.labelsize']=10
mpl.rcParams['ytick.labelsize']=10
mpl.rcParams['ytick.major.pad']=8
mpl.rcParams['font.size']=12


In [ ]:
#@title plank blackbody function
def planck_bb_wm2srum(lam, T):
    c1 = 1.191041e8 # W / m^2 / sr / um^4
    c2 = 1.4387752e4 # um K
    return c1/(np.exp(c2/lam/T) - 1)/lam**5 # W/m^2/sr/um

In [ ]:
#@title load etaloning data
rr0 = np.load('')
test_spectrum = rr0[20,100,:]
fig,ax = plt.subplots(1,1,figsize=(7,3),dpi=180)
ax.plot(test_spectrum)

In [ ]:
def fit_polycal(wl, px, order=3):
    return np.polyfit(wl, px, order)

def cal(wl, calcoefs):

    return np.polyval(calcoefs, wl)

def cal_inv(wl, calcoefs):
    wls = np.linspace(wl.min(), wl.max(), 1000)
    inv_cal = np.polyfit(cal(wls, calcoefs), wls, 5)
    def inv(x):
        return np.polyval(inv_cal, x)
    return inv

# wavelength calibration given the calibration coefficients
class WlCal:
    def __init__(lam0, px0, cal_coeffs):
        self.lam0 = lam0
        self.px0 = px0
        self.cal_coeffs = cal_coeffs

        self.px_to_wl = cal_inv(self.lam0, self.px0, self.cal_coeffs)
        self.wl_to_px = cal(self.lam0, self.px0, self.cal_coeffs)

    def px_to_wl(self, px):
        return self.px_to_wl(px)


# load calibration data
vnir_cal = WlCal(vnir_lam0, vnir_px0, vnir_cal_coeffs)
px = np.arange(test_spectrum.shape[0])
wl = vnir_cal.px_to_wl1d(px, x0)
fig,ax = plt.subplots(1,1,figsize=(7,3),dpi=180)
ax.plot(wl, test_spectrum)

In [ ]:
#@title wavelength to pixel calibration
vnir_cal_coeffs = np.array([ -81.58508159, 1083.4032634,   512.80926573])
swir_cal_coeffs = np.array([  3.77102206,  4.05665725, 978.92156902, 629.2644374 ])
mwir_prism_cal_coeffs = np.array([  3.27044779,  19.00826691 , 83.48343561 , 627.11166944])

vnir_lam0 = 0.657
vnir_px0 = 512

mwir_cal_px2lam = functools.partial(cal, cal_coeffs=mwir_prism_cal_coeffs)
vnir_cal_px2lam = functools.partial(cal, cal_coeffs=vnir_cal_coeffs)
swir_cal_px2lam = functools.partial(cal, cal_coeffs=swir_cal_coeffs)

In [ ]:
vnir_cal_coeffs = np.array([ -81.58508159, 1083.4032634,   512.80926573])
def fit_polycal(wl, px, order=3):
    return np.polyfit(wl, px, order)

def cal(wl, calcoefs):
    return np.polyval(calcoefs, wl)

def cal_inv(wl, calcoefs):
    wls = np.linspace(wl.min(), wl.max(), 1000)
    inv_cal = np.polyfit(cal(wls, calcoefs), wls, 5)
    def inv(x):
        return np.polyval(inv_cal, x)
    return inv

fig, ax = plt.subplots(1,1,figsize=(7,3),dpi=180)

lam = np.linspace(0.3, 1.1, 1000)
px = cal(lam, vnir_cal_coeffs)
ax.plot(lam, px, '.')

invcal = cal_inv(lam, vnir_cal_coeffs)

ax.plot(invcal(px), px, 'r-')

In [ ]:

def edge_filter(x, low, high, lam_width=1):
    filt = np.zeros_like(x)
    filt[(x > low) & (x < high)] = 1.0
    dlam = x[1] - x[0]
    width = int(lam_width / dlam)*5
    gk = np.exp(-np.linspace(-width,width,2*width+1)**2/width**2)
    gk = gk / gk.sum()
    filt = np.convolve(filt, gk, mode='same')
    return filt

def bb_bp(lam, temp, low, high, width):
    l = planck_bb_wm2srum(lam, temp)
    l = l*edge_filter(lam, low, high, width)
    return l

px = np.arange(0,rr0.shape[2],1)
# lam = px_to_lam(px, 500, 1000, 0)
lam = invcal(px)
lam = px
sig = rr0[100,100,:]

# lam = np.linspace(0.3, 1.1, 1000)

bbt = bb_bp(lam, 2000, 0.35, 0.9, 0.003)


def etaloning(lam, amp_coeffs, phase_coeffs):
    amp = cheb.chebval(lam, amp_coeffs)
    phase = cheb.chebval(lam, phase_coeffs)
    return amp*np.cos(phase), amp, phase



fig, ax = plt.subplots(1,1,figsize=(4,4),dpi=200)


ax.plot(lam, sig, color='k', lw=1.0, alpha=0.5)
# ax.plot(lam, bbt, color='r', alpha=0.5)

# amp_coeffs = [1, 1, 1]
# phase_coeffs = [1, 80]
# phase_coeffs = [1, 1, 1]
# s, amp, phase = etaloning(lam, amp_coeffs, phase_coeffs)